In [1]:
# Import required libraries
import numpy as np
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt
import joblib

In [2]:
#Missing value handle
data = pd.read_csv("Coustmer_churn.csv")
print(data.isnull().sum())

customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64


In [3]:
#Droped unneccessary column.
data.drop(['customerID'] , axis=1 , inplace=True)

In [4]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
binary_col = ["Partner", 'Dependents', 'PhoneService', 'MultipleLines', 'OnlineSecurity',
               'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV',
               'StreamingMovies', 'PaperlessBilling', 'Churn', 'gender']
for col in binary_col:
    data[col] = data[col].replace({'No phone service': 'No', 'No internet service': 'No'})  # Safe replace
    data[col] = le.fit_transform(data[col])

mul_col = ['InternetService', 'Contract', 'PaymentMethod']
for col in mul_col:
    data[col] = le.fit_transform(data[col])

data['TotalCharges'] = pd.to_numeric(data['TotalCharges'], errors='coerce').fillna(0)

print(data.isnull().sum())
print(data)

gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64
      gender  SeniorCitizen  Partner  Dependents  tenure  PhoneService  \
0          0              0        1           0       1             0   
1          1              0        0           0      34             1   
2          1              0        0           0       2             1   
3          1              0        0           0      45             0   
4          0              0        0           0       2             1   
...      ...            ...      ...         ...     ...           ...   
7038       1              0  

In [5]:
x = data[['gender','SeniorCitizen','Partner','Dependents','tenure','PhoneService','MultipleLines','InternetService','OnlineSecurity','OnlineBackup','DeviceProtection','TechSupport','StreamingTV','StreamingMovies','Contract','PaperlessBilling','PaymentMethod','MonthlyCharges','TotalCharges']]
y = data['Churn']

from sklearn.model_selection import train_test_split
x_train , x_test , y_train, y_test = train_test_split(x , y ,test_size=0.33 , stratify=y , random_state=42)

from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=42)
x_train_sm, y_train_sm = smote.fit_resample(x_train, y_train)


In [6]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(class_weight='balanced', random_state=42)
parameter = {
    "n_estimators": [100],
    "criterion": ['gini', 'entropy', 'log_loss'],
    "max_depth": [5, 10, ],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4],
    "max_features": ['sqrt', 'auto', 'log2'],
}
from sklearn.model_selection import GridSearchCV
cv = GridSearchCV(rf , param_grid=parameter , cv = 5 , scoring='f1' , n_jobs=-1)
cv.fit(x_train_sm , y_train_sm)


c:\Users\ansik\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py:528: FitFailedWarning: 
270 fits failed out of a total of 810.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
129 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\ansik\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\ansik\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py", line 1382, in wrapper
    estimator._validate_params()
  File "c:\Users\ansik\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base

GridSearchCV(cv=5,
             estimator=RandomForestClassifier(class_weight='balanced',
                                              random_state=42),
             n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy', 'log_loss'],
                         'max_depth': [5, 10],
                         'max_features': ['sqrt', 'auto', 'log2'],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [100]},
             scoring='f1')

In [7]:
print(cv.best_params_)
y_pred = cv.predict(x_test)
print(y_pred)
from sklearn.metrics import accuracy_score , classification_report
print(accuracy_score(y_test , y_pred))
print(classification_report(y_test,y_pred))

{'criterion': 'gini', 'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
[0 0 0 ... 0 0 0]
0.7750537634408602
              precision    recall  f1-score   support

           0       0.88      0.80      0.84      1708
           1       0.56      0.71      0.63       617

    accuracy                           0.78      2325
   macro avg       0.72      0.75      0.73      2325
weighted avg       0.80      0.78      0.78      2325



In [8]:
import joblib 
joblib.dump(cv , 'churn_model.pkl')

['churn_model.pkl']